In [1]:
import requests
import json
import re
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [2]:
#create a list of the makeup tabs we want to loop through and 
#a seperate list for the max amount of pages that are available for each tab

search_list = ['foundation-makeup', 'blush', 'eyeshadow', 'eyeliner', 'moisturizer-skincare', 'face-wash-facial-cleanser', 'shampoo-sulfate-free-shampoo', 'conditioner-hair' ]
max_page = [4, 2, 5, 3, 9, 5, 4, 3]

In [3]:
link_list = []
type_list = []

#searches through the tabs and pages to generate a list of all products and information

for search, page in zip(search_list, max_page):
    print(search)
    print("------")
    
    counter = 1
    
    while counter <= page:   
        base_url = f"https://www.sephora.com/shop/{search}?currentPage={counter}"
        source = requests.get(base_url).text
        soup = BeautifulSoup(source, 'lxml')
        
        link = soup.find_all('a', class_='css-ix8km1')
        for url in link:
            link_list.append(url.get('href'))
            type_list.append(search)
        
        counter += 1

foundation-makeup
------
blush
------
eyeshadow
------
eyeliner
------
moisturizer-skincare
------
face-wash-facial-cleanser
------
shampoo-sulfate-free-shampoo
------
conditioner-hair
------


In [ ]:
url_list = []

for link in link_list:
    url_list.append("%20".join(link.split()))
    
# pprint(url_list)

In [5]:
#extracting info on details per product but still failing, it is
#returning the ingredients for some products (not really sure why though)
brand_list = []
price_list = []
rating_list = []
product_list = []
ingredients_list = []
detail_list = []

for url in url_list:
    #'css-pz80c5'
    
    base_url = "https://www.sephora.com" + url
    source = requests.get(base_url).text
    soup = BeautifulSoup(source, 'lxml')
    
    details_ingredients = soup.find_all('div', class_='css-1rny024')
    ingredients_list.append(details_ingredients[2].text)   
    
    detail_list.append(details_ingredients[0].text)
            
    brand = soup.find_all('a', class_='css-es084o')
    brand_list.append(brand[0].text)
            
    product = soup.find_all('span', class_='css-0')
    product_list.append(product[0].string)
            
    price = soup.find_all('div', class_='css-slwsq8')
    price_list.append(price[0].string)
            
    rating = soup.find_all('div', class_= 'css-ychh9y')
    rating_list.append(rating[0]['aria-label'])

# pprint(detail_list)

In [6]:
# print(brand_list[0:5])
# print(price_list[0:5])
# print(rating_list[0:5])
# print(product_list[0:5])
# print(ingredients_list[0:5])
# print(detail_list[0:5])

In [7]:
# sample = []
# base_url = "https://www.sephora.com" + url_list[417]
# source = requests.get(base_url).text
# soup = BeautifulSoup(source, 'lxml')
    
# brand = soup.find_all('a', class_='css-es084o')
# sample.append(brand[0].text)
# sample

In [9]:
# price_array = []
# for price in price_list:
#     price_array.append(str(price))

dict = {'Brand': brand_list,
        'Product': product_list,
        'Product Type': type_list,
        'Price': price_list,
        'Rating': rating_list,
        'Price': price_list,
        'Details': detail_list,
        'Ingredients': ingredients_list,
        'URL': url_list
       }

for prod in product_list:
    if prod == None:
        product_list.remove(prod)
        
sephora_df = pd.DataFrame(dict)

sephora_df

#I want to label the type of product but theres an uneven number of products per type
# the amount is equivalent to 12 * max_page for each type
#ex: foundation, there should be 12*4 = 48 foundations

,Brand,Product,Product Type,Price,Rating,Details,Ingredients,URL
0,Estée Lauder,Double Wear Stay-in-Place Foundation,foundation-makeup,$43.00,4.5 stars,What it is: A 24-hour liquid foundation with a...,"Get more information about shipping rates, sch...",/product/double-wear-stay-in-place-makeup-P378...
1,FENTY BEAUTY by Rihanna,Pro Filt'r Soft Matte Longwear Foundation,foundation-makeup,$35.00,4 stars,"What it is: A soft matte, long-wear foundation...","Water, Dimethicone, Talc, Peg-10 Dimethicone, ...",/product/pro-filtr-soft-matte-longwear-foundat...
2,Armani Beauty,Luminous Silk Foundation,foundation-makeup,$64.00,4.5 stars,What it is: An award-winning oil-free foundati...,-Micro-Fil Technology: Provides a translucent ...,/product/luminous-silk-foundation-P393401?icid...
3,MAKE UP FOR EVER,Ultra HD Invisible Cover Foundation,foundation-makeup,$43.00,4 stars,What it is: An iconic foundation that provides...,-Hyaluronic Spheres: Formulated to look fresh ...,/product/ultra-hd-invisible-cover-foundation-P...
4,Bite Beauty,Changemaker Supercharged Micellar Foundation,foundation-makeup,$39.50,4.5 stars,"What it is: A clean, long-wearing foundation w...",-Maqui Berry: An antioxidant-rich ingredient ...,/product/bite-beauty-changemaker-supercharged-...
...,...,...,...,...,...,...,...,...
415,IGK,Bad & Bougie Amla Oil Deep Repair Conditioner,conditioner-hair,$25.00,3.5 stars,Which hair type is it good for?✔ Straight✔ Wav...,"Water, Cetearyl Alcohol, Glycerin, Butyrosperm...",/product/bad-bougie-amla-oil-deep-repair-condi...
416,Good Dye Young,Color Kind Conditioner,conditioner-hair,$26.00,4.5 stars,What it is: A conditioner power-packed with in...,"Water, Cetearyl Alcohol, Behentrimonium Chlori...",/product/color-kind-conditioner-P437304?icid2=...
417,IGK,Thirsty Girl Coconut Milk Anti-Frizz Conditioner,conditioner-hair,$25.00,4.5 stars,What it is: A smoothing conditioner with cocon...,"Water, Cetearyl Alcohol, Cetyl Esters, Amodime...",/product/thirsty-girl-coconut-milk-anti-frizz-...
418,Moroccanoil,Curl Cleansing Conditioner,conditioner-hair,$32.00,4.5 stars,Which hair type is it good for?✔ Wavy✔ Curly✔ ...,"-Argan Oil: Rich in tocopherols (Vitamin E), ...",/product/curl-cleansing-conditioner-P419711?ic...
